In [ ]:
onnx_model_path = "./model/model.onnx"

ExecutionProvider="QNNExecutionProvider"

### Data Processor

In [ ]:
def _fix_winrt_runtime():
    """This function removes the msvcp140.dll from the winrt-runtime package.
    So it does not cause issues with other libraries.
    """
    from importlib import metadata
    from pathlib import Path
    site_packages_path = Path(str(metadata.distribution('winrt-runtime').locate_file('')))
    dll_path = site_packages_path / 'winrt' / 'msvcp140.dll'
    if dll_path.exists():
        dll_path.unlink()
            
def _get_ep_paths() -> dict[str, str]:
    from winui3.microsoft.windows.applicationmodel.dynamicdependency.bootstrap import (
        InitializeOptions,
        initialize
    )
    import winui3.microsoft.windows.ai.machinelearning as winml
    eps = {}
    with initialize(options = InitializeOptions.ON_NO_MATCH_SHOW_UI):
        catalog = winml.ExecutionProviderCatalog.get_default()
        providers = catalog.find_all_providers()
        for provider in providers:
            provider.ensure_ready_async().get()
            eps[provider.name] = provider.library_path
            # DO NOT call provider.try_register in python. That will register to the native env.
    return eps

def _regsiter_executino_providers_to_onnxruntime():
    import onnxruntime as ort

    paths = _get_ep_paths()
    for item in paths.items():
        print(f"----register ort ep---- {item[0]} {item[1]}")
        ort.register_execution_provider_library(item[0], item[1])

_fix_winrt_runtime()
_regsiter_executino_providers_to_onnxruntime()

In [ ]:
from transformers import CLIPProcessor

processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K")

### Model Inference with ORT-QNN

In [ ]:
import onnxruntime as ort
import torch
import numpy as np

def add_ep_for_device(session_options, ep_name, device_type, ep_options=None):
    ep_devices = ort.get_ep_devices()
    for ep_device in ep_devices:
        if ep_device.ep_name == ep_name and ep_device.device.type == device_type:
            print(f"Adding {ep_name} for {device_type}")
            session_options.add_provider_for_devices([ep_device], {} if ep_options is None else ep_options)


session_options = ort.SessionOptions()

add_ep_for_device(session_options, ExecutionProvider, ort.OrtHardwareDeviceType.NPU)

vision_model = ort.InferenceSession(
    onnx_model_path, # a model wirh QNN EPContext nodes
    sess_options=session_options,
)

def get_image_embedding(image):
    inputs = processor(images=image, return_tensors="np")
    output = vision_model.run(None, { "pixel_values": inputs["pixel_values"] })
    return torch.from_numpy(output[0])

def calculate_score(emb_1, emb_2):
    emb_1 /= torch.norm(emb_1, dim=-1, keepdim=True)
    emb_2 /= torch.norm(emb_2, dim=-1, keepdim=True)
    return torch.matmul(emb_1, emb_2.T) * 100.0

# Get source embedding and calculate the similarity score for each target
# We need to process one by one because to static quantization, we fixed the batch size to 1
def ask(source, targets):
    source_emb = get_image_embedding(source)
    for i, target in enumerate(targets):
        target_emb = get_image_embedding(target)
        score = calculate_score(source_emb, target_emb)
        print(f"Similarity score of image {i}：{score.item()}")


### Play with Samples

In [ ]:
import requests
from PIL import Image

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
url = "http://images.cocodataset.org/train2017/000000208833.jpg"
image1 = Image.open(requests.get(url, stream=True).raw)
image1

In [ ]:
url = "http://images.cocodataset.org/train2017/000000125690.jpg"
image2 = Image.open(requests.get(url, stream=True).raw)
image2

In [ ]:
ask(image, [image1, image2])